# Visualizing Solar Radiation Data
Kevin and I wanted to visualize solar radiation data for the Phillips Academy campus to find potential locations to conduct solar projects. Specifically, we have both been involved in Andover Solar Initiative on campus and we wanted to use our data visualization to prove the efficacy of solar panels in certain locations on campus. However, after some research we quickly realized that solar radiation data was quite hard to measure on such a small scale. Instead, we found [this really informative database](https://nsrdb.nrel.gov/) that contained solar radiation data created from the Physical Solar Model for the entire world in 4 km by 4 km chunks. 

It was at this point, we decided to move up to larger scale and visualize solar radiation data for Massachusetts. Later, this would turn out to be almost the perfect size, any larger scale would have been too much to handle data-wise (e.g. New England, US).

Fortunately for us, NSRDB also had a Python API that we could use to collect their data. Thus, we followed [their instructions](https://nsrdb.nrel.gov/api-instructions) and collected the relevant data. This also built on our knowledge of `requests` and REST API from one of the earlier homework assignments.

In [32]:
import pandas as pd
import numpy as np
import os, sys
import math

import requests
import json

### API format
We quickly learned the format for querying the yearly data for one 4 km x 4 km grid. 
- `lat, lon` is simply the GPS coordinate
- `api_key` was obtained from [signing up here](https://developer.nrel.gov/signup/)
- `names` is an array of the years specified (the API only returns a year's worth data at a time)
- `leap_day` is a boolean specifying whether to include 366th day (if it even exists)
- `interval` is whether the data will be collected in half-hour or hourly intervals
- `email` was required for some reason in the [API documentation](https://developer.nrel.gov/docs/solar/nsrdb/psm_data_download/)
Then, we could simply declare the url string and make the request to download the csv.

In [8]:
# location of example
lat, lon = 33.2164, -97.1292
# my api key
api_key = 'YQBXn6Yx0YNsfzBBsP96vSxSRie7kLT7QxzXXVaJ'
# attributes to extract 
attributes = 'ghi,dhi,dni,wind_speed_10m_nwp,surface_air_temperature_nwp,solar_zenith_angle'
# Choose year of data
year = 2011
# Set leap year to true or false. True will return leap day data if present, false will not.
leap_year = 'false'
# Set time interval in minutes, i.e., '30' is half hour intervals. Valid intervals are 30 & 60.
interval = '60'
utc = 'false'
# Your email address
your_email = 'eyou@andover.edu'
# Declare url string
url = 'http://developer.nrel.gov/api/solar/nsrdb_0512_download.csv?wkt=POINT({lon}%20{lat})&names={year}&leap_day={leap}&interval={interval}&utc={utc}&email={email}&api_key={api}&attributes={attr}'.format(year=year, lat=lat, lon=lon, leap=leap_year, interval=interval, utc=utc, email=your_email, api=api_key, attr=attributes)
# Return just the first 2 lines to get metadata:
info = pd.read_csv(url, nrows=1)

### Examining Data

As you can see below, the Physical Solar Model was extremely, extremely descriptive. In fact, too descriptive, as each file size was 512 kb, meaning that we would need to greatly reduce the dimensions of our data.

In [9]:
# view metadata
info

,Source,Location ID,City,State,Country,Latitude,Longitude,Time Zone,Elevation,Local Time Zone,...,Cloud Type 10,Cloud Type 11,Cloud Type 12,Fill Flag 0,Fill Flag 1,Fill Flag 2,Fill Flag 3,Fill Flag 4,Fill Flag 5,Version
0,NSRDB,680780,-,-,-,33.21,-97.14,-6,203,-6,...,Unknown,Dust,Smoke,NaN,Missing Image,Low Irradiance,Exceeds Clearsky,Missing CLoud Properties,Rayleigh Violation,v2.0.1


In [10]:
# Return all but first 2 lines of csv to get data:
df = pd.read_csv(url, skiprows=2)

# Set the time index in the pandas dataframe:
df = df.set_index(pd.date_range('1/1/{yr}'.format(yr=year), freq=interval+'Min', periods=525600/int(interval)))

# take a look
print('shape:',df.shape)
df.head()

shape: (8760, 11)


,Year,Month,Day,Hour,Minute,GHI,DHI,DNI,Wind Speed,Temperature,Solar Zenith Angle
2011-01-01 00:00:00,2011,1,1,0,30,0,0,0,3.563944,2.057062,169.805752
2011-01-01 01:00:00,2011,1,1,1,30,0,0,0,3.790218,1.397638,163.663120
2011-01-01 02:00:00,2011,1,1,2,30,0,0,0,3.894154,0.594415,152.161329
2011-01-01 03:00:00,2011,1,1,3,30,0,0,0,3.812119,-0.274512,139.752955
2011-01-01 04:00:00,2011,1,1,4,30,0,0,0,3.677367,-1.067603,127.218790


## Collecting Data
Now that we could grab a year's worth of data for one location, we needed a way to grab a year's worth of data for everywhere in Massachusetts. However, no such dataset of latitudes and longitudes existed. Instead, we looked towards using our geojson data for the state of MA and simply filling in the coordinates using the [shapely library](https://pypi.python.org/pypi/Shapely) to check whether the GIS coordinate was inside of our geojson Masschusetts.

To narrow down our selection, we satisfied the following requirements
- since our model only stored data for 4 km by 4 km pixels, our latitude step size became 4 km / 111 km (average km per degree of latitude) = **0.036036 degrees**
- due to the geometry of the Earth, the length of one degree of longitude actually depends on latitude, so we first approximated the longitude step size by `step_size =  4 km / deg_length = cos(latitude) * latitudinal length at equator (110.567 km)` which we rounded to **0.048667 degrees**

Then, we picked two GIS points that completely enclosed MA in a box and simply ran a for loop to collect every single GIS point in Massachusetts that corresponded to a location roughly 4 km from its nearest neighbor to get a collection of grid points.

In [13]:
# import the necessary library
from shapely.geometry import shape, Point

# load GeoJSON file containing the svg 
with open('assets/ma.json') as f:
    js = json.load(f)
    
# Function that returns whether the point located at the given coordinate is within the MA geojson
def place_in_MA(lat, long):
    point = Point(long, lat)
    for feature in js['features']:
        polygon = shape(feature['geometry'])
        return polygon.contains(point)

# Set initial and our boundary conditions 
curLat, curLong = 42.9, -73.527100
finalLat, finalLong = 41.235685, -69.835693
latStep, longStep=0.036036, 0.048667

# list to store all of our points
points = []

# iterate through each possible grid point between the boundary conditions
while curLat > finalLat:
    tempLong = curLong
    while tempLong < finalLong:
        if place_in_MA(curLat, tempLong):
            points.append([curLat, tempLong])
        tempLong = tempLong + longStep
    curLat = curLat - latStep

# load the collected points into a pandas dataframe
points = pd.DataFrame.from_records(points, columns=["lat", "long"])
points.head()
# export the points to a csv for later use in display.
points.to_csv("ma_coords.csv", index=False)

# Collecting Solar Data

Finally, we put everything together and used our list of latitude and longitude values to request our solar data for each point to compile a massive collection of files. We initially wanted to start out from 2015 and work our back to 1999, but quickly we realized that we could not accomodate for the sheer volume of the data and only stuck with the most recent year (2015). 
In addition, to minimize the dimensions of data, we only requested the following:
- global horizontal irradiation (ghi): Modeled solar radiation on a horizontal surface received from the sky. Measured in watts/m^2
- direct normal irradiation (dni): Modeled solar radiation obtained from the direction of the sun. Measured in watts/m^2
- surface air temperature: Taken from [MERRA](https://gmao.gsfc.nasa.gov/reanalysis/MERRA/) Measured in degrees Celsius.
- solar zenith angle: Angle between the sun and the zenith (e.g. zenith angle is 90 degrees when sun is directly above an observer) Measured in degrees. 

In [15]:
api_key = 'YQBXn6Yx0YNsfzBBsP96vSxSRie7kLT7QxzXXVaJ'
attributes = 'ghi,dni,surface_air_temperature_nwp,solar_zenith_angle'
years = [2015]
interval = '60'
utc = 'false'
your_email = 'eyou@andover.edu'
# read in the coordinates for which we wanted to query the solar radiation data
coords = pd.read_csv("ma_coords.csv")
# loop through all of the coordinates for each year specified
for i in range(len(years)):
    for j in range(len(coords)):  
        url = 'http://developer.nrel.gov/api/solar/nsrdb_0512_download.csv?wkt=POINT({lon}%20{lat})&names={year}&interval={interval}&utc={utc}&email={email}&api_key={api}&attributes={attr}'.format(year=years[i], lat=coords['lat'][j], lon=coords['long'][j], interval=interval, utc=utc, email=your_email, api=api_key, attr=attributes)
        # HTTP GET request to NSRDB
        response = requests.get(url) 
        # write the data to our data folder with all of the necessary metadata as the file name
        with open(os.path.join("data", "data_{}_{}_{}.csv".format(years[i], coords['lat'][j], coords['long'][j])), 'wb') as f:
            f.write(response.content)

# Cleaning Data Up
Now that we had retrieved all of the data, we needed to clean it up and compress it so that it was easily accessible. Just to give you a sense of how large our data was: 513 kb x 1293 points = ~ 660 mb. 
In addition, this is what it looked like:

| Year | Month | Day | Hour | Minute | GHI | DNI | Temperature | Solar Zenith Angle
--|--|--|--|--|--|--|--|--|--
2011-01-01 00:00:00	| 2011 | 1 | 1 | 0 | 30 | 0 | 0 | 2.057062 | 169.805752
2011-01-01 01:00:00 | 2011 | 1 | 1 | 1 | 30	| 0 | 0 | 1.397638 | 163.663120
2011-01-01 02:00:00	| 2011 | 1 | 1 | 2 | 30 | 0 | 0 | 0.594415 | 152.161329
2011-01-01 03:00:00	| 2011 | 1 | 1 | 3 | 30	| 0 | 0 | -0.274512 | 139.752955
2011-01-01 04:00:00	| 2011 | 1 | 1 | 4 | 30	| 0 | 0 | -1.067603 | 127.218790

To reduce both the volume, as well as display meaningfull information, we decided to change the following information:
- restrict 24 hours to 6:00 am --- 6:00 pm: this reduces the volume by almost half (13/24 = ~1/2)
- remove hour and minute values and use index values (i.e. integer division and modulo by 13)
- reduce 365 days to 52 weeks: this eliminates year, month, day as we kept track of days through purely index values (i.e. multiplication by 13)
- remove solar zenith angle: we felt that this was not as useful in plotting pure solar energy
- remove DNI: if you recall earlier, DNI is the amount of radiation energy in watts/m^2 directly facing the sun. However, GHI included all radiation energy incoming, including the DNI
- round the temperature to two decimal places: floats are huge
Doing so, we were able to go from 513 kb per file to around 52 kb

In [30]:
# specify hours from 6 - 18 inclusive
hours = [i for i in range(6, 19)]

# specify only two data values
data = pd.DataFrame(columns=['GHI','Temperature'])

# loop through all files 
for file in os.listdir('data_2015'):
    # ensure that we only capture the right data files
    if file == '.DS_Store':
        continue
    print(file)
    # We skip the first two rows of metadata, as everything important is already in file name
    df = pd.read_csv('data_2015/'+file, skiprows=2)
    # We drop the unnecessary column data completely
    df = df.drop('Month', axis=1)
    df = df.drop('Day', axis=1)
    df = df.drop('Year', axis=1)
    df = df.drop('Minute', axis=1)
    df = df.drop('Solar Zenith Angle', axis=1)
    df = df.drop('DNI', axis=1)
    # We filter the hours
    df = df.loc[df['Hour'].isin(hours)]
    # we drop the hours now
    df = df.drop('Hour', axis=1)
    # We have to reset the indices, since we removed rows
    df = df.reset_index(drop=True)
    
    # We add on the weekly information
    week_df = pd.DataFrame()
    i = 0
    # the number of entries 
    weekEntries = 7 * len(hours)
    while i < len(df):
        for j in range(len(hours)):
            if len(hours) + i < len(df):
                week_df = week_df.append(df.loc[i+j])
        i = i + weekEntries
    df = week_df
    df = df.reset_index(drop=True)

    for i in range(len(df)):
        # We round temperature to two decimal places
        df.set_value(i, 'Temperature', float("{0:.1f}".format(df['Temperature'][i])))
    # we add the current dataframe to our overall dataframe
    data = data.append(df)
data.to_csv('data.csv', index=False)

data_2015_41.5048429999999_-70.99641600000028.csv
data_2015_41.5048429999999_-71.04508300000028.csv
data_2015_41.540878999999904_-70.60708000000031.csv
data_2015_41.540878999999904_-70.65574700000032.csv
data_2015_41.540878999999904_-70.9477490000003.csv
data_2015_41.540878999999904_-70.99641600000028.csv
data_2015_41.540878999999904_-71.04508300000028.csv
data_2015_41.540878999999904_-71.09375000000027.csv
data_2015_41.57691499999991_-70.46107900000034.csv
data_2015_41.57691499999991_-70.50974600000032.csv
data_2015_41.57691499999991_-70.55841300000033.csv
data_2015_41.57691499999991_-70.60708000000031.csv
data_2015_41.57691499999991_-70.99641600000028.csv
data_2015_41.57691499999991_-71.04508300000028.csv
data_2015_41.57691499999991_-71.09375000000027.csv
data_2015_41.6129509999999_-70.26641100000036.csv
data_2015_41.6129509999999_-70.46107900000034.csv
data_2015_41.6129509999999_-70.50974600000032.csv
data_2015_41.6129509999999_-70.55841300000033.csv
data_2015_41.6129509999999_-70.6

data_2015_41.86520299999993_-71.33708500000024.csv
data_2015_41.901238999999926_-69.97440900000039.csv
data_2015_41.901238999999926_-70.07174300000038.csv
data_2015_41.901238999999926_-70.55841300000033.csv
data_2015_41.901238999999926_-70.60708000000031.csv
data_2015_41.901238999999926_-70.65574700000032.csv
data_2015_41.901238999999926_-70.70441400000031.csv
data_2015_41.901238999999926_-70.75308100000032.csv
data_2015_41.901238999999926_-70.8017480000003.csv
data_2015_41.901238999999926_-70.8504150000003.csv
data_2015_41.901238999999926_-70.89908200000029.csv
data_2015_41.901238999999926_-70.9477490000003.csv
data_2015_41.901238999999926_-70.99641600000028.csv
data_2015_41.901238999999926_-71.04508300000028.csv
data_2015_41.901238999999926_-71.09375000000027.csv
data_2015_41.901238999999926_-71.14241700000026.csv
data_2015_41.901238999999926_-71.19108400000027.csv
data_2015_41.901238999999926_-71.23975100000025.csv
data_2015_41.901238999999926_-71.28841800000025.csv
data_2015_41.901

data_2015_42.081418999999954_-71.9210890000002.csv
data_2015_42.081418999999954_-71.96975600000017.csv
data_2015_42.081418999999954_-72.01842300000017.csv
data_2015_42.081418999999954_-72.06709000000015.csv
data_2015_42.081418999999954_-72.11575700000016.csv
data_2015_42.081418999999954_-72.16442400000015.csv
data_2015_42.081418999999954_-72.21309100000015.csv
data_2015_42.081418999999954_-72.26175800000014.csv
data_2015_42.081418999999954_-72.31042500000014.csv
data_2015_42.081418999999954_-72.35909200000012.csv
data_2015_42.081418999999954_-72.40775900000013.csv
data_2015_42.081418999999954_-72.45642600000012.csv
data_2015_42.081418999999954_-72.50509300000013.csv
data_2015_42.081418999999954_-72.55376000000011.csv
data_2015_42.081418999999954_-72.60242700000009.csv
data_2015_42.081418999999954_-72.6510940000001.csv
data_2015_42.081418999999954_-72.6997610000001.csv
data_2015_42.081418999999954_-72.74842800000009.csv
data_2015_42.081418999999954_-72.79709500000007.csv
data_2015_42.08

data_2015_42.18952699999996_-71.38575200000024.csv
data_2015_42.18952699999996_-71.43441900000023.csv
data_2015_42.18952699999996_-71.48308600000024.csv
data_2015_42.18952699999996_-71.53175300000022.csv
data_2015_42.18952699999996_-71.58042000000022.csv
data_2015_42.18952699999996_-71.6290870000002.csv
data_2015_42.18952699999996_-71.6777540000002.csv
data_2015_42.18952699999996_-71.7264210000002.csv
data_2015_42.18952699999996_-71.7750880000002.csv
data_2015_42.18952699999996_-71.82375500000019.csv
data_2015_42.18952699999996_-71.87242200000018.csv
data_2015_42.18952699999996_-71.9210890000002.csv
data_2015_42.18952699999996_-71.96975600000017.csv
data_2015_42.18952699999996_-72.01842300000017.csv
data_2015_42.18952699999996_-72.06709000000015.csv
data_2015_42.18952699999996_-72.11575700000016.csv
data_2015_42.18952699999996_-72.16442400000015.csv
data_2015_42.18952699999996_-72.21309100000015.csv
data_2015_42.18952699999996_-72.26175800000014.csv
data_2015_42.18952699999996_-72.3104

data_2015_42.297634999999964_-71.48308600000024.csv
data_2015_42.297634999999964_-71.53175300000022.csv
data_2015_42.297634999999964_-71.58042000000022.csv
data_2015_42.297634999999964_-71.6290870000002.csv
data_2015_42.297634999999964_-71.6777540000002.csv
data_2015_42.297634999999964_-71.7264210000002.csv
data_2015_42.297634999999964_-71.7750880000002.csv
data_2015_42.297634999999964_-71.82375500000019.csv
data_2015_42.297634999999964_-71.87242200000018.csv
data_2015_42.297634999999964_-71.9210890000002.csv
data_2015_42.297634999999964_-71.96975600000017.csv
data_2015_42.297634999999964_-72.01842300000017.csv
data_2015_42.297634999999964_-72.06709000000015.csv
data_2015_42.297634999999964_-72.11575700000016.csv
data_2015_42.297634999999964_-72.16442400000015.csv
data_2015_42.297634999999964_-72.21309100000015.csv
data_2015_42.297634999999964_-72.26175800000014.csv
data_2015_42.297634999999964_-72.31042500000014.csv
data_2015_42.297634999999964_-72.35909200000012.csv
data_2015_42.2976

data_2015_42.405742999999966_-72.06709000000015.csv
data_2015_42.405742999999966_-72.11575700000016.csv
data_2015_42.405742999999966_-72.16442400000015.csv
data_2015_42.405742999999966_-72.21309100000015.csv
data_2015_42.405742999999966_-72.26175800000014.csv
data_2015_42.405742999999966_-72.31042500000014.csv
data_2015_42.405742999999966_-72.35909200000012.csv
data_2015_42.405742999999966_-72.40775900000013.csv
data_2015_42.405742999999966_-72.45642600000012.csv
data_2015_42.405742999999966_-72.50509300000013.csv
data_2015_42.405742999999966_-72.55376000000011.csv
data_2015_42.405742999999966_-72.60242700000009.csv
data_2015_42.405742999999966_-72.6510940000001.csv
data_2015_42.405742999999966_-72.6997610000001.csv
data_2015_42.405742999999966_-72.74842800000009.csv
data_2015_42.405742999999966_-72.79709500000007.csv
data_2015_42.405742999999966_-72.84576200000008.csv
data_2015_42.405742999999966_-72.89442900000007.csv
data_2015_42.405742999999966_-72.94309600000007.csv
data_2015_42.4

data_2015_42.513850999999974_-72.35909200000012.csv
data_2015_42.513850999999974_-72.40775900000013.csv
data_2015_42.513850999999974_-72.45642600000012.csv
data_2015_42.513850999999974_-72.50509300000013.csv
data_2015_42.513850999999974_-72.55376000000011.csv
data_2015_42.513850999999974_-72.60242700000009.csv
data_2015_42.513850999999974_-72.6510940000001.csv
data_2015_42.513850999999974_-72.6997610000001.csv
data_2015_42.513850999999974_-72.74842800000009.csv
data_2015_42.513850999999974_-72.79709500000007.csv
data_2015_42.513850999999974_-72.84576200000008.csv
data_2015_42.513850999999974_-72.89442900000007.csv
data_2015_42.513850999999974_-72.94309600000007.csv
data_2015_42.513850999999974_-72.99176300000006.csv
data_2015_42.513850999999974_-73.04043000000007.csv
data_2015_42.513850999999974_-73.08909700000005.csv
data_2015_42.513850999999974_-73.13776400000005.csv
data_2015_42.513850999999974_-73.18643100000004.csv
data_2015_42.513850999999974_-73.23509800000005.csv
data_2015_42.5

data_2015_42.62195899999999_-72.11575700000016.csv
data_2015_42.62195899999999_-72.16442400000015.csv
data_2015_42.62195899999999_-72.21309100000015.csv
data_2015_42.62195899999999_-72.26175800000014.csv
data_2015_42.62195899999999_-72.31042500000014.csv
data_2015_42.62195899999999_-72.35909200000012.csv
data_2015_42.62195899999999_-72.40775900000013.csv
data_2015_42.62195899999999_-72.45642600000012.csv
data_2015_42.62195899999999_-72.50509300000013.csv
data_2015_42.62195899999999_-72.55376000000011.csv
data_2015_42.62195899999999_-72.60242700000009.csv
data_2015_42.62195899999999_-72.6510940000001.csv
data_2015_42.62195899999999_-72.6997610000001.csv
data_2015_42.62195899999999_-72.74842800000009.csv
data_2015_42.62195899999999_-72.79709500000007.csv
data_2015_42.62195899999999_-72.84576200000008.csv
data_2015_42.62195899999999_-72.89442900000007.csv
data_2015_42.62195899999999_-72.94309600000007.csv
data_2015_42.62195899999999_-72.99176300000006.csv
data_2015_42.62195899999999_-73.0

In [41]:
pd.read_csv('data_2015_clean/data.csv')

,GHI,Temperature
0,0.0,-1.3
1,28.0,-1.5
2,172.0,-1.3
3,310.0,-0.7
4,407.0,-0.1
5,451.0,0.4
6,438.0,0.9
7,265.0,1.1
8,210.0,1.3
9,74.0,1.5
